# Auto-Roll MVP (Notebook / CLI)

Minimal text-only TTRPG DM assistant that runs in a single cell. The assistant:

1. Receives player input.
2. Lets an Ollama model decide whether a roll is required.
3. Automatically rolls using the existing dice engine.
4. Returns in-character narration without any extra UI fluff.


## 1. Setup

This cell pulls requirements and wires in the dice engine plus the Ollama client. The model is queried deterministically (`temperature=0`).


In [32]:
from __future__ import annotations

import json
import os
from dataclasses import dataclass
from typing import Any, Dict, List, Tuple

import ollama

# Locate dice engine implementation
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / 'DiceTool_BasicMCP').exists():
    for parent in PROJECT_ROOT.parents:
        candidate = parent.resolve()
        if (candidate / 'DiceTool_BasicMCP').exists():
            PROJECT_ROOT = candidate
            break
    else:
        raise RuntimeError('DiceTool_BasicMCP directory not found. Run from project root or adjust path.')

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from DiceTool_BasicMCP import server as dice_server  # type: ignore

OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'llama3.1:8b')
SYSTEM_PROMPT = """
You are the Orchestrator for a TTRPG DM.

Always output exactly one JSON object with keys "ooc" and "ic".

ooc.event ∈ {"NONE","ROLL_REQUEST","ROLL_RESOLVE","STATE_CHANGE"}.

For "ROLL_REQUEST":
- Include "actor", "check.skill", "check.dc" (Easy 10 | Medium 13–15 | Hard 17–20), and "adv" ("none" | "advantage" | "disadvantage").
- Do not narrate here.

For "ROLL_RESOLVE":
- Use only the provided "roll_result" with fields {"total", "dc", "nat"}.
- Compute:
    diff = total - dc
    degree = 5 if nat20 or diff ≥ 10
    degree = 4 if 5 ≤ diff ≤ 9
    degree = 3 if 0 ≤ diff ≤ 4
    degree = 2 if −4 ≤ diff ≤ −1
    degree = 1 if nat1 or diff ≤ −5
- Include "outcome" ("crit_success" | "success" | "mixed" | "fail" | "crit_fail") and "degree".

For "STATE_CHANGE":
- Include "ops" with {"op": "inc" | "set", "path", "value"}.

The "ic" field must be one or two sentences of in-character narration — no dice numbers, no OOC text.

Never invent rolls.

Use deterministic decoding (temperature = 0).
""".strip()


# Mutable game state used by the orchestrator
GAME_STATE: Dict[str, Any] = {
    "scene": {"id": "tavern", "lighting": "dim", "environment": "indoors", "description": "A cozy tavern filled with the aroma of roasted meat and the sound of lively chatter."},
    "actors": {
        "Jake": {
            "skills": {"Pursuasion": 12, "Perception": 16, "Arcana": 20, "Dexterity": 12, "Constitution": 10, "Intelligence": 18, "Wisdom": 16, "Charisma": 8, "Strength": 14, "Stealth": 8, "Intimidation": 12, "Deception": 8},
            "inventory": {"gold": 150, "items": ["short sword", "leather armor", "healing potion"]},
            "hp": 22,
            "conditions": [],
        },
    },
}


## 2. Helpers

Utility functions for model I/O, dice rolling, logging, and game-state updates.


In [33]:
def ollama_chat(messages: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Query Ollama deterministically and return the assistant message as a dict."""
    response = ollama.chat(
        model=OLLAMA_MODEL,
        messages=messages,
        options={"temperature": 0.0}
    )
    message = response.message.model_dump(exclude_none=True)
    message.setdefault('content', '')
    return message


def get_json_response(conversation: List[Dict[str, Any]], max_attempts: int = 2) -> Dict[str, Any]:
    """Call the model until it returns valid JSON with keys ooc/ic or raise."""
    correction = {
        'role': 'system',
        'content': 'Return only valid JSON with keys "ooc" and "ic".'
    }
    for attempt in range(max_attempts):
        assistant_msg = ollama_chat(conversation)
        conversation.append(assistant_msg)
        raw = assistant_msg.get('content', '').strip()
        try:
            payload = json.loads(raw)
            if not isinstance(payload, dict) or 'ooc' not in payload or 'ic' not in payload:
                raise ValueError('Missing ooc/ic keys')
            return payload
        except Exception as exc:
            if attempt == max_attempts - 1:
                raise RuntimeError(f'Model failed to produce valid JSON: {exc}') from exc
            conversation.append(correction)
    raise RuntimeError('Unreachable')


def mcp_dice_roll(skill_mod: int, dc: int, advantage: str) -> Tuple[Dict[str, Any], Dict[str, Any]]:
    policy_map = {
        'advantage': 'advantage.v1',
        'disadvantage': 'disadvantage.v1',
        'none': 'core.v1',
    }
    policy_key = policy_map.get((advantage or 'none').lower(), 'core.v1')
    modifier = int(skill_mod)
    formula = f"1d20{modifier:+d}"
    roll = dice_server.engine.run(formula, policy_key)
    breakdown = roll['breakdown']
    kept = breakdown.get('kept') or breakdown['rolls']
    kept_val = kept[0] if kept else breakdown['rolls'][0]
    nat20 = kept_val == 20
    nat1 = kept_val == 1
    payload = {
        'total': int(roll['total']),
        'dc': int(dc),
        'nat': bool(nat20),
    }
    detail = {
        'total': int(roll['total']),
        'rolls': breakdown['rolls'],
        'kept': kept_val,
        'modifier': modifier,
        'policy': policy_key,
        'nat20': nat20,
        'nat1': nat1,
    }
    return payload, detail


def apply_state_changes(game_state: Dict[str, Any], ops: List[Dict[str, Any]]) -> None:
    for op in ops:
        path = op.get('path')
        operation = op.get('op')
        value = op.get('value')
        if not path or operation not in {'inc', 'set'}:
            print(f"[STATE] Ignored invalid op: {op}")
            continue
        segments = path.split('.')
        cursor = game_state
        for segment in segments[:-1]:
            if isinstance(cursor, dict) and segment in cursor:
                cursor = cursor[segment]
            else:
                print(f"[STATE] Path not found: {path}")
                break
        else:
            key = segments[-1]
            if operation == 'set':
                cursor[key] = value
                print(f"[STATE] Set {path} -> {value}")
            elif operation == 'inc':
                try:
                    if isinstance(value, str):
                        value = value.strip()
                        if value.lstrip('-').isdigit():
                            value_num = int(value)
                        else:
                            value_num = float(value)
                    else:
                        value_num = value
                    if not isinstance(value_num, (int, float)):
                        raise TypeError('Increment value must be numeric')
                    current = cursor.get(key, 0) if isinstance(cursor, dict) else 0
                    cursor[key] = current + value_num
                    print(f"[STATE] Inc {path} by {value_num} -> {cursor[key]}")
                except Exception as exc:
                    print(f"[STATE] Failed inc {path}: {exc}")


def build_turn_messages(game_state: Dict[str, Any], player_text: str) -> List[Dict[str, Any]]:
    state_blob = json.dumps({'game_state': game_state}, separators=(',', ':'))
    turn_blob = json.dumps({'game_state': game_state, 'player_input': player_text}, separators=(',', ':'))
    return [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'system', 'content': state_blob},
        {'role': 'user', 'content': turn_blob},
    ]


def neutral_response() -> str:
    return 'The moment hangs in silence; nothing more happens yet.'


## 3. Turn Runner

This function executes a full auto-rolled turn: model request ➝ optional dice roll ➝ resolution ➝ state updates.


In [34]:
def run_turn(player_text: str) -> None:
    print(f"Player: {player_text}")
    conversation = build_turn_messages(GAME_STATE, player_text)
    try:
        first_payload = get_json_response(conversation)
    except RuntimeError as exc:
        print(f"DM: I need a moment ({exc}).")
        return

    ooc = first_payload.get('ooc', {})
    event = ooc.get('event', 'NONE')

    if event == 'ROLL_RESOLVE':
        conversation.append({'role': 'system', 'content': 'A roll result has not been provided yet. Issue a ROLL_REQUEST before resolving.'})
        try:
            first_payload = get_json_response(conversation)
        except RuntimeError as exc:
            print(f"DM: The outcome is hazy ({exc}).")
            return
        ooc = first_payload.get('ooc', {})
        event = ooc.get('event', 'NONE')

    if event == 'ROLL_REQUEST':
        actor = ooc.get('actor')
        check = ooc.get('check', {})
        skill = check.get('skill')
        try:
            dc = int(check.get('dc', 10))
        except Exception:
            dc = 10
        adv = (check.get('adv') or 'none').lower()

        print(f"[ROLL] Request -> actor={actor}, skill={skill}, dc={dc}, adv={adv}")

        actor_data = GAME_STATE.get('actors', {}).get(actor, {}) if actor else None
        if not actor_data or not skill or skill not in actor_data.get('skills', {}):
            print(f"[ROLL] Invalid request for actor={actor} skill={skill}; ignoring.")
            print(f"DM: {neutral_response()}")
            return

        modifier = int(actor_data['skills'][skill])
        roll_payload, detail = mcp_dice_roll(modifier, dc, adv)
        roll_payload['check'] = skill

        rolls_str = ', '.join(str(r) for r in detail['rolls'])
        mod_str = f"{modifier:+d}"
        special = ' [NAT20!]' if detail['nat20'] else (' [NAT1!]' if detail['nat1'] else '')
        print(f"[ROLL] {actor} rolls {skill} vs DC {dc} ({adv}); rolls [{rolls_str}] kept {detail['kept']} {mod_str} -> total {detail['total']}{special}")

        conversation.append({'role': 'system', 'content': json.dumps({'roll_result': roll_payload}, separators=(',', ':'))})

        try:
            second_payload = get_json_response(conversation)
        except RuntimeError as exc:
            print(f"DM: The outcome is hazy ({exc}).")
            return

        result_meta = second_payload.get('ooc', {}).get('result', {})
        if result_meta:
            outcome = result_meta.get('outcome')
            degree = result_meta.get('degree')
            total = result_meta.get('total')
            dc_val = result_meta.get('dc')
            print(f"[ROLL] Outcome -> total {total} vs DC {dc_val} => {outcome} (degree {degree})")

        apply_ops_if_any(second_payload)
        ic_line = second_payload.get('ic', '').strip() or neutral_response()
        print(f"DM: {ic_line}")
        return

    if event not in {'NONE', 'STATE_CHANGE'}:
        print(f"[WARN] Unexpected event '{event}'. Treating as narration.")

    apply_ops_if_any(first_payload)
    ic_line = first_payload.get('ic', '').strip() or neutral_response()
    print(f"DM: {ic_line}")


def apply_ops_if_any(payload: Dict[str, Any]) -> None:
    ooc = payload.get('ooc', {})
    ops = ooc.get('ops')
    if ops:
        apply_state_changes(GAME_STATE, ops)


In [35]:
!pip install -q ollama


In [36]:
!ollama --version

ollama version is 0.12.6


In [37]:
!ollama pull llama3.1:8b

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest 
pulling 667b0c1932bc: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 4.9 GB                         
pulling 948af2743fc7: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.5 KB                         
pulling 0ba8f0e314b4: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  12 KB                         
pulling 56bb8bd477a5: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�   96 B                         
pulling 455f34728c9b: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  487 B                         
verifying sha256 digest 
writing manifest 
success 


## 4. CLI Loop

Type commands; enter `quit` or `exit` to stop. Logs show roll requests, results, and state changes.


In [ ]:
print("Auto-Roll DM test. Type 'quit' to exit.")

while True:
    try:
        player_line = input('> ').strip()
    except (EOFError, KeyboardInterrupt):
        print('Exiting.')
        break
    if not player_line:
        continue
    if player_line.lower() in {'quit', 'exit'}:
        print('Goodbye.')
        break
    run_turn(player_line)


Auto-Roll DM test. Type 'quit' to exit.


>  I grab my sword and attack whoever is nearest to me.


Player: I grab my sword and attack whoever is nearest to me.


[10/24/25 16:31:44] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=728096;file://C:\Python311\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=231188;file://C:\Python311\Lib\site-packages\httpx\_client.py#1025\1025]8;;\

[ROLL] Request -> actor=Jake, skill=None, dc=10, adv=none
[ROLL] Invalid request for actor=Jake skill=None; ignoring.
DM: The moment hangs in silence; nothing more happens yet.


...

[10/24/25 16:34:36] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=843436;file://C:\Python311\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=543469;file://C:\Python311\Lib\site-packages\httpx\_client.py#1025\1025]8;;\

[ROLL] Request -> actor=Jake, skill=Strength, dc=10, adv=none
[ROLL] Jake rolls Strength vs DC 10 (none); rolls [13] kept 13 +14 -> total 27


[10/24/25 16:34:42] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=876566;file://C:\Python311\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=628358;file://C:\Python311\Lib\site-packages\httpx\_client.py#1025\1025]8;;\

[10/24/25 16:34:53] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=904356;file://C:\Python311\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=738221;file://C:\Python311\Lib\site-packages\httpx\_client.py#1025\1025]8;;\

DM: You grip your short sword tightly, preparing to strike at the nearest person.
